In [1]:
from work.unet.data_functions import *

Using TensorFlow backend.


In [2]:
train_path = r'D:\Clarifruit\cherry_stem\data\unet_data\train'
test_path = r'D:\Clarifruit\cherry_stem\data\unet_data\test'
test_aug_path = os.path.join(test_path, 'aug')

target_size = (256, 256)
modes_dict = {'grayscale': 1, 'rgb': 3}

color_mode = 'rgb'

x_folder_name = 'image'
y_folder_name = 'label'

x_prefix = 'image'
y_prefix = 'label'

weights_file_name = 'unet_cherry_stem.hdfs5'
input_size = (*target_size, modes_dict[color_mode])

data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

In [3]:

train_gen = clarifruit_train_generator(batch_size=2,
                                       train_path=train_path,
                                       image_folder=x_folder_name,
                                       mask_folder=y_folder_name,
                                       aug_dict=data_gen_args,
                                       image_color_mode=color_mode,
                                       mask_color_mode='grayscale',
                                       image_save_prefix=x_prefix,
                                       mask_save_prefix=y_prefix,
                                       flag_multi_class=False,
                                       num_class=2,
                                       save_to_dir=None,
                                       target_size=target_size,
                                       seed=1)

test_gen = clarifruit_train_generator(batch_size=2,
                                      train_path=test_path,
                                      image_folder=x_folder_name,
                                      mask_folder=y_folder_name,
                                      aug_dict=data_gen_args,
                                      image_color_mode=color_mode,
                                      mask_color_mode='grayscale',
                                      image_save_prefix=x_prefix,
                                      mask_save_prefix=y_prefix,
                                      flag_multi_class=False,
                                      num_class=2,
                                      save_to_dir=None,
                                      target_size=target_size,
                                      seed=1)



In [4]:
model = unet(input_size=input_size)
model_checkpoint = ModelCheckpoint(weights_file_name, monitor='loss', verbose=1, save_best_only=True)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\Clarifruit\cherry_stem\work\unet\model.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


In [5]:
model.fit_generator(train_gen, steps_per_epoch=100, epochs=3, callbacks=[model_checkpoint])

Epoch 1/3
Found 133 images belonging to 1 classes.
Found 133 images belonging to 1 classes.
100/100 [==============================] - 48s 483ms/step - loss: 0.1622 - acc: 0.9844

Epoch 00001: loss improved from inf to 0.16259, saving model to unet_cherry_stem.hdfs5
Epoch 2/3
100/100 [==============================] - 26s 258ms/step - loss: 0.0710 - acc: 0.9844

Epoch 00002: loss improved from 0.16259 to 0.07098, saving model to unet_cherry_stem.hdfs5
Epoch 3/3
100/100 [==============================] - 26s 258ms/step - loss: 0.0515 - acc: 0.9847

Epoch 00003: loss improved from 0.07098 to 0.05166, saving model to unet_cherry_stem.hdfs5


In [6]:
model.evaluate_generator(test_gen, steps=30, callbacks=[model_checkpoint], verbose=1)

Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.
30/30 [==============================] - 3s 100ms/step


[0.04449215615010363, 0.985897258176642]

In [7]:
test = custom_generator(batch_size=1,
                        train_path=test_path,
                        folder=x_folder_name,
                        aug_dict={**data_gen_args,**dict(rescale=1./255)},
                        save_prefix=x_prefix,
                        color_mode=color_mode,
                        save_to_dir=test_aug_path,
                        target_size=target_size,
                        seed=1)
results = model.predict_generator(generator=test, steps=30, verbose=1)
saveResult(test_aug_path,test,results)

Found 57 images belonging to 1 classes.
30/30 [==============================] - 2s 55ms/step


d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\38357-13707.png.jpg_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\38357-27009.png.jpg_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\38357-47985.png.jpg_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\38357-62566.png.jpg_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
d:\clarifruit\cherry_stem\ch